
## **Notebook Overview — Fixed-Effects Decomposition and Bootstrap**

 **Purpose:**
 This notebook estimates a fixed-effects (FE) model to decompose SES-related differences in essay scores  
 into *content*, *style*, and *other* components, and optionally computes bootstrap confidence intervals.

**How to Use:**
 - **Run up to Cell 3** if you only need the **decomposition dataset**:  
   This will fit the FE model once, append the fixed effects (`fe_essay`, `fe_k`) and residuals (`u`)  
   to each observation, and save the full augmented dataset as a CSV file.  
   --> Output file: `decomp_rows_with_fe.csv`

 - **Run all cells (1–7)** if you also want to **compute bootstrap confidence intervals**:  
   This will perform 500 clustered bootstrap replications at the essay level (resampling essays  
   with all rewrites), re-estimate the FE model each time, and produce percentile-based 95% CIs  
   for the total, content, style, and others decomposition metrics.

 **Outputs:**
 - `decomp_rows_with_fe.csv` --> Row-level data with FE components for all essays and rewrites.  
 - `summary_df` --> Compact table with decomposition point estimates and bootstrap 95% CIs (Both in .csv and .tex formats).

### **Cell 1 — Imports & Configuration**

**Purpose:**
- Import all core Python libraries used throughout the notebook.  
- Prepare tools for data handling (`pandas`, `numpy`), model estimation (`pyfixest`), and progress visualization (`tqdm`).  
- Include utilities for parallel computation (`joblib`) and statistical modeling (`statsmodels`).  
- Initialize a global random number generator (`rng`) with a fixed seed to ensure reproducibility of bootstrap samples.


In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
from joblib import Parallel, delayed
import statsmodels.api as sm
from pyfixest.estimation import feols

# Reproducibility
rng = np.random.default_rng(42)

In [2]:
out_path = "../data/results/"

df_path = "../data/results/data_no_desc_scored_final.csv"

### **Cell 2 — Load & Basic Preparation**

**Purpose:**
- Load the scored dataset containing essay-level and rewrite-level information.  
- Sort rows by `essay_id` and `k` so each essay’s rewrites appear sequentially, ensuring consistent ordering for later fixed-effect estimation and bootstrapping.  
- Reset the index to maintain a clean 0–N row index after sorting.


In [3]:
df = pd.read_csv(df_path, encoding="ISO-8859-1", index_col=0)

df = df.sort_values(["essay_id","k"]).reset_index(drop=True)

### **Cell 3 — Fit Fixed-Effects Model and Extract Baseline Estimates**

**Purpose:**
- Estimate a two-way fixed-effects model using `pyfixest` to control for essay-specific and prompt-specific variation.  
- Model specification: `score_high_full ~ 1 | essay_id + k`.  
- Retrieve the estimated fixed effects from the model output (`fes`):  
  - `fe_essay` → captures essay-level (content) differences.  
  - `fe_k` → captures prompt-level (context) effects.  
- Map these fixed effects back to each observation in the DataFrame for later decomposition.  
- Compute residuals `u = score_high_full – fe_essay`, representing the style component once essay-level effects are removed.


In [4]:
# FE regression
res = feols("score_high_full ~ 1 | essay_id + k", data=df)
print(res.summary)

fes = res.fixef()
fe_essay_map = fes["C(essay_id)"]  #
fe_k_map = fes["C(k)"]

# Map back to rows
df["fe_essay"] = df["essay_id"].map(fe_essay_map)
df["fe_k"] = df["k"].map(fe_k_map)

# Style residual (after removing essay FE only, as in your code)
df["u"] = df["score_high_full"] - df["fe_essay"]

# Compute diff column
df["diff"] = df["score_high_full"] - df["score_low_full"]

# Save DataFrame with FEs and residuals
df.to_csv(out_path + "decomp_rows_with_fe.csv", index=False)
print(f"Saved augmented dataset - {out_path}decomp_rows_with_fe.csv")

functools.partial(<function summary at 0x1555dc720>, models=[<pyfixest.estimation.feols_.Feols object at 0x158f01fd0>])
Saved augmented dataset - ../data/results/decomp_rows_with_fe.csv


In [5]:
# Define a small utility function to compute conditional means while safely handling missing values (`NaN`). 

def mean_mask(s: pd.Series, mask: pd.Series) -> float:
    s2 = s[mask & s.notna()]
    return float(s2.mean()) if len(s2) else np.nan


### **Cell 4 — Decomposition**

**Purpose:**
- Compute the SES gap and its decomposition into **content**, **style**, and **others** using the FE outputs.  
- Restrict the decomposition to **k == 0** for comparability.  
- Return both **absolute gaps** and **shares** (each component divided by the total gap).

**Steps:**
 - **A) Total gap (k==0):**  
   - Group 0 = high-SES `score_high_full` where `low_SES==0`.  
   - Group 1 = low-SES  `score_low_full`  where `low_SES==1`.  
   - `total_gap = mean(group 0) – mean(group 1)`.
 - **B) Content gap (k==0):**  
   - Compare `fe_essay` means between SES groups; essay FE proxies content.  
   - `content_gap = mean(fe_essay | high-SES) – mean(fe_essay | low-SES)`.
 - **C) Style gap (k==0):**  
   - Use residual `u = score_high_full – fe_essay` as a style proxy.  
   - `style_gap = mean(u | high-SES) – mean(u | low-SES)`.
 - **D) Others gap (k==0, within high-SES):**  
   - Difference between `score_high_full` and `score_low_full` (same SES group).  
   - `others_gap = mean(score_high_full) – mean(score_low_full)` for `low_SES==0`.

**Outputs:**
 - Dictionary with absolute gaps: `total_gap`, `content_gap`, `style_gap`, `others_gap`.  
 - Shares: `share_content`, `share_style`, `share_others` (component / total_gap).  
 - `point_est` stores the baseline (non-bootstrap) decomposition for the full sample.

In [6]:
def compute_decomposition_point_estimates(df_in: pd.DataFrame):
    df_ = df_in.copy()

    # --- A) total gap (k==0)
    mA = (df_["low_SES"]==0) & (df_["k"]==0) & df_["score_high_full"].notna()
    mB = (df_["low_SES"]==1) & (df_["k"]==0) & df_["score_low_full"].notna()
    tmp_vals = pd.Series(np.nan, index=df_.index)
    tmp_vals[mA] = df_.loc[mA, "score_high_full"].astype(float)
    tmp_vals[mB] = df_.loc[mB, "score_low_full"].astype(float)

    group = pd.Series(np.nan, index=df_.index)
    group[mA] = 0
    group[mB] = 1

    m0 = (group==0)
    m1 = (group==1)

    mean0 = mean_mask(tmp_vals, m0)
    mean1 = mean_mask(tmp_vals, m1)
    total_gap = mean0 - mean1

    # --- B) content gap (k==0), use essay FE
    mK0 = (df_["k"]==0)
    content_mean0 = mean_mask(df_["fe_essay"], mK0 & (df_["low_SES"]==0))
    content_mean1 = mean_mask(df_["fe_essay"], mK0 & (df_["low_SES"]==1))
    content_gap = content_mean0 - content_mean1

    # --- C) style gap via residual u = score_high_full - fe_essay
    u_mean0 = mean_mask(df_["u"], mK0 & (df_["low_SES"]==0))
    u_mean1 = mean_mask(df_["u"], mK0 & (df_["low_SES"]==1))
    style_gap = u_mean0 - u_mean1

    # --- D) others gap (your original "diff" within high-SES at k==0)
    m_last = (df_["low_SES"]==0) & (df_["k"]==0)
    sh_mean = mean_mask(df_["score_high_full"], m_last)
    sl_mean = mean_mask(df_["score_low_full"],  m_last)
    others_gap = sh_mean - sl_mean

    return {
        "total_gap": total_gap,
        "content_gap": content_gap,
        "style_gap": style_gap,
        "others_gap": others_gap,
        "share_content": content_gap/total_gap if pd.notna(total_gap) and total_gap!=0 else np.nan,
        "share_style": style_gap/total_gap   if pd.notna(total_gap) and total_gap!=0 else np.nan,
        "share_others": others_gap/total_gap  if pd.notna(total_gap) and total_gap!=0 else np.nan,
    }

point_est = compute_decomposition_point_estimates(df)
point_est


{'total_gap': 0.6572251073631734,
 'content_gap': 0.518098210274891,
 'style_gap': 0.10459843024325868,
 'others_gap': 0.06186198976259494,
 'share_content': 0.7883116522336344,
 'share_style': 0.15915160433068948,
 'share_others': 0.0941260293764323}


### **Cell 5 — Bootstrap (Clustered at essay_id; Parallelized, 500 Reps, 2.5–97.5% CI)**

**Purpose:**
- Estimate uncertainty for the decomposition metrics using **clustered bootstrapping** at the essay level.  
- Each bootstrap sample resamples essays (clusters) **with replacement**, keeping all their rewrites together.  
- The process re-fits the fixed-effects model on each resample, computes the decomposition again,  
  and aggregates results across 500 iterations to form **95% confidence intervals**.

**Steps:**
1. Reset DataFrame index and precompute a dictionary mapping each `essay_id` to its row indices  
2. Define the helper function `one_rep(seed)` that performs a **single bootstrap iteration**:  
   - Samples `essay_id`s with replacement (cluster bootstrap).  
   - Extracts the corresponding rows efficiently via `.iloc`.  
   - Re-fits the FE model (`score_high_full ~ 1 | essay_id + k`).  
   - Computes essay and prompt fixed effects, residuals, and decomposition metrics.  
3. Generate a list of independent random seeds (one per iteration) to ensure reproducibility  
   across parallel workers.  
4. Use **Joblib’s `Parallel`** and **`delayed`** utilities to run all 500 bootstrap replications concurrently  
   (`n_jobs=-1` automatically uses all available CPU cores).  
5. Convert the resulting list of dictionaries into a single DataFrame (`boot_df_stats`),  
   where each row corresponds to one bootstrap iteration and each column to a decomposition metric.  
6. Compute percentile-based confidence intervals (2.5% and 97.5%) for every statistic,  
   storing them in `ci_bounds` as a compact dictionary of lower–upper tuples.


In [7]:
df = df.reset_index(drop=True)

essay_to_idx = df.groupby("essay_id").indices
essay_ids = np.array(list(essay_to_idx.keys()))
n_clusters = len(essay_ids)


def one_rep(seed: int) -> dict:
    """
    Performs ONE bootstrap iteration:
    1. Samples essay_ids with replacement.
    2. Collects all their rows from df.
    3. Fits the fixed-effects model.
    4. Computes the decomposition metrics.
    5. Returns a dictionary with the results.
    """

    rng_local = np.random.default_rng(seed)

    sampled = rng_local.choice(essay_ids, size=n_clusters, replace=True)

    idx = np.concatenate([essay_to_idx[eid] for eid in sampled])

    df_b = df.iloc[idx]

    res_b = feols("score_high_full ~ 1 | essay_id + k", data=df_b)

    fes_b = res_b.fixef()
    fe_essay_b = fes_b["C(essay_id)"]  # essay-level effects
    fe_k_b = fes_b["C(k)"] # prompt-level effects

    df_b = df_b.copy()

    df_b["fe_essay"] = df_b["essay_id"].map(fe_essay_b)
    df_b["fe_k"] = df_b["k"].map(fe_k_b)

    df_b["u"] = df_b["score_high_full"] - df_b["fe_essay"]

    # Compute total/content/style/others gaps and shares on this sample
    # Returns a dict like {'total_gap': x, 'content_gap': y, 'style_gap': z, ...}
    return compute_decomposition_point_estimates(df_b)


# --- Parallel execution setup ---
B = 500  # number of bootstrap iterations

# Create an array of random seeds (one per bootstrap replication)
# This ensures each parallel worker uses a unique, reproducible seed
seeds = rng.integers(0, 2**32 - 1, size=B, dtype=np.uint64)

print(f"Running {B} bootstrap iterations in parallel...")

# Run the bootstrap in parallel using all available CPU cores
# - n_jobs=-1  = use all cores
# - backend='loky' = safe multiprocessing backend for heavy tasks
# - delayed(one_rep)(int(s)) = schedules one_rep(seed) for each seed
# - tqdm(seeds)  = adds progress bar
results = Parallel(n_jobs=-1, backend="loky")(
    delayed(one_rep)(int(s)) for s in tqdm(seeds)
)

# After all parallel jobs finish, 'results' is a list of dicts
boot_df_stats = pd.DataFrame(results)

# --- Compute bootstrap confidence intervals ---
ci_bounds = {}
for col in boot_df_stats.columns:
    low, high = np.nanpercentile(boot_df_stats[col], [2.5, 97.5])
    ci_bounds[col] = (low, high)

ci_bounds


Running 500 bootstrap iterations in parallel...


100%|██████████| 500/500 [01:47<00:00,  4.66it/s]


{'total_gap': (np.float64(0.6320802301989173), np.float64(0.6837142444898432)),
 'content_gap': (np.float64(0.49303931617708774),
  np.float64(0.5451547111003249)),
 'style_gap': (np.float64(0.09617602620540146),
  np.float64(0.11219516620095764)),
 'others_gap': (np.float64(0.05783373141660024),
  np.float64(0.0662591759843586)),
 'share_content': (np.float64(0.7742281514369145),
  np.float64(0.8026585149995842)),
 'share_style': (np.float64(0.14666320021384152),
  np.float64(0.17290491273081907)),
 'share_others': (np.float64(0.0874803281322524),
  np.float64(0.100877995749596))}

### **Cell 7 — Summary Table (Point Estimates + Bootstrap Confidence Intervals)**

**Purpose:**
- Combine the **baseline decomposition point estimates** (from Cell 5)  
with the **bootstrap confidence intervals** (from Cell 6) into one concise summary table.  

In [8]:
summary_rows = []
for name, pe in point_est.items():
    lo, hi = ci_bounds[name]
    summary_rows.append({
        "stat": name,
        "point_estimate": round(pe, 3),
        "ci_2.5": round(lo, 3),
        "ci_97.5": round(hi, 3)
    })

summary_df = pd.DataFrame(summary_rows)
summary_df


,stat,point_estimate,ci_2.5,ci_97.5
0,total_gap,0.657,0.632,0.684
1,content_gap,0.518,0.493,0.545
2,style_gap,0.105,0.096,0.112
3,others_gap,0.062,0.058,0.066
4,share_content,0.788,0.774,0.803
5,share_style,0.159,0.147,0.173
6,share_others,0.094,0.087,0.101


In [9]:
# save summary_df to csv
summary_df.to_csv(out_path + "decomposition_bootstrap_summary.csv", index=False)

# save summary_df to latex
summary_df.to_latex(out_path + "decomposition_bootstrap_summary.tex", index=False)